# Aula de Ciência de Dados para Devs: Limpeza e Preparação de Dados

**Bem-vindo(a) à Parte 2: A Prática!**

Se na teoria tudo parece fazer sentido, é na prática que o conhecimento realmente se fixa. Nesta aula, você vai atuar como um detetive de dados. Vamos pegar um arquivo de cadastro de usuários de um e-commerce fictício (`usuarios_sujo.csv`), que está cheio de problemas comuns do mundo real, e vamos aplicar técnicas sistemáticas para limpá-lo e organizá-lo.

Ao final, teremos um dataset íntegro, confiável e pronto para a próxima fase: a análise exploratória de dados, onde poderemos extrair insights valiosos.

**Ferramentas que usaremos:**
- **Python:** Nossa linguagem de programação principal.
- **Pandas:** A biblioteca essencial para manipulação e análise de dados em Python. Pense nela como uma planilha superpoderosa que você controla com código.
- **NumPy:** Usada pelo Pandas por baixo dos panos, é fundamental para operações numéricas eficientes.
- **Faker:** Para gerar dados fictícios e criar nosso próprio dataset "sujo".

### Nosso Fluxo de Trabalho

Para nos guiar, seguiremos um fluxo de trabalho estruturado. Pense nisso como um mapa que nos levará do caos à ordem:

**1. Fonte de Dados Brutos (`usuarios_sujo.csv`)**
   - Nosso ponto de partida. Um arquivo com dados realistas, porém problemáticos.
     
**2. Carregamento e Inspeção Inicial (Profiling)**
   - Carregar os dados em um DataFrame e usar ferramentas de diagnóstico para identificar os problemas.
     
**3. Ciclo de Limpeza (Transformação)**
   - Esta é a fase iterativa onde aplicamos as correções:
     - Tratar Valores Nulos
     - Corrigir Tipos de Dados
     - Remover Duplicatas
     - Padronizar Dados Categóricos
     
**4. Validação**
   - Verificar se a limpeza foi bem-sucedida e se os dados agora fazem sentido.
     
**5. Saída de Dados Limpos (`usuarios_limpo.csv`)**
   - Salvar nosso trabalho em um novo arquivo, pronto para ser usado em análises futuras.

---
## 1. Setup do Ambiente e Geração dos Dados

Primeiro, vamos importar as bibliotecas necessárias. Em seguida, vamos criar nosso próprio arquivo `usuarios_sujo.csv`. Isso garante que todos tenham exatamente o mesmo ponto de partida e que o notebook seja autocontido.

**Tipos de problemas que vamos introduzir:**
- **Valores Faltantes:** `email` e `valor_ultima_compra` terão valores nulos (`NaN`).
- **Tipos de Dados Incorretos:** `valor_ultima_compra` será uma string (object) com símbolos e vírgulas, e as colunas de data serão strings.
- **Formatos Inconsistentes:** A coluna `data_cadastro` terá múltiplos formatos de data (ex: 'YYYY-MM-DD', 'DD/MM/YYYY').
- **Dados Categóricos Não Padronizados:** A coluna `estado` terá variações como 'SP', 'São Paulo' e 'sao paulo'.
- **Linhas Duplicadas:** Inseriremos algumas linhas completamente duplicadas.

In [ ]:
%pip install faker
# Importando as bibliotecas
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime

# Inicializando o Faker para gerar dados em português
fake = Faker('pt_BR')

# Função para gerar os dados sujos
def gerar_dados_sujos(num_usuarios=200):
    dados = []

    # Formatos de data que vamos misturar
    formatos_data = ['%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y', '%d-%b-%Y']

    # Variações para os estados
    estados_sp = ['SP', 'São Paulo', 'sao paulo']
    estados_rj = ['RJ', 'Rio de Janeiro', 'rio de janeiro']
    outros_estados = ['MG', 'PR', 'BA', 'SC']

    for i in range(num_usuarios):
        # Introduzindo valores faltantes (NaN) de forma aleatória
        email = fake.email() if random.random() > 0.1 else np.nan # 10% de chance de email nulo
        valor_compra = round(random.uniform(10, 1000), 2) if random.random() > 0.15 else np.nan # 15% de chance de valor nulo

        # Formatando o valor da compra como string com inconsistências
        if pd.notna(valor_compra):
            valor_compra_str = f"R$ {valor_compra:.2f}".replace('.', ',')
        else:
            # Adicionando outras strings não numéricas para sujar mais
            valor_compra_str = np.nan if random.random() > 0.3 else 'Não informado'

        # Escolhendo um formato de data aleatório
        formato_escolhido = random.choice(formatos_data)
        data_cadastro = fake.date_between(start_date='-2y', end_date='today').strftime(formato_escolhido)

        # Escolhendo um estado com inconsistências
        if i % 4 == 0:
            estado = random.choice(estados_sp)
        elif i % 7 == 0:
            estado = random.choice(estados_rj)
        else:
            estado = random.choice(outros_estados)

        dado = {
            'user_id': fake.uuid4(),
            'nome': fake.name(),
            'email': email,
            'data_cadastro': data_cadastro,
            'cidade': fake.city(),
            'estado': estado,
            'valor_ultima_compra': valor_compra_str,
            'data_ultimo_login': fake.date_time_between(start_date='-30d', end_date='now')
        }
        dados.append(dado)

    df = pd.DataFrame(dados)

    # Introduzindo linhas duplicadas
    duplicatas = df.sample(n=15, random_state=42)
    df_final = pd.concat([df, duplicatas]).reset_index(drop=True)

    return df_final

# Gerar e salvar o arquivo CSV
df_sujo = gerar_dados_sujos()
df_sujo.to_csv('usuarios_sujo.csv', index=False)

print("Arquivo 'usuarios_sujo.csv' gerado com sucesso!")
print(f"Total de linhas: {len(df_sujo)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.1 MB/s eta 0:00:00
Arquivo 'usuarios_sujo.csv' gerado com sucesso!
Total de linhas: 215


---
## 2. Passo a Passo do Exercício Prático

Agora começa a nossa missão! Vamos carregar o arquivo `usuarios_sujo.csv` que acabamos de criar e seguir nosso fluxo de trabalho para limpá-lo passo a passo.

### 2.1 Carregamento dos Dados

Usaremos a função `pd.read_csv()` do Pandas para ler nosso arquivo e carregá-lo em uma estrutura de dados chamada **DataFrame**. Pense no DataFrame como uma tabela ou planilha dentro do Python.

In [ ]:
# Carregando o arquivo CSV para um DataFrame
df_usuarios = pd.read_csv('usuarios_sujo.csv')

# Exibindo as dimensões do DataFrame (linhas, colunas)
print(f"O dataset possui {df_usuarios.shape[0]} linhas e {df_usuarios.shape[1]} colunas.")

O dataset possui 215 linhas e 8 colunas.


### 2.2 Inspeção Inicial (Data Profiling)

Antes de sair corrigindo tudo, precisamos entender a "cena do crime". O Data Profiling é o processo de investigar o dataset para entender sua estrutura, qualidade e conteúdo. É aqui que identificamos os problemas.

#### Usando `.head()`, `.tail()` e `.sample()` para Amostragem

Essas funções nos permitem "espiar" os dados de diferentes ângulos:
- `.head(n)`: Mostra as primeiras `n` linhas (o padrão é 5).
- `.tail(n)`: Mostra as últimas `n` linhas (o padrão é 5).
- `.sample(n)`: Mostra uma amostra aleatória de `n` linhas, útil para ter uma visão imparcial do dataset.

In [ ]:
print("--- Primeiras 5 linhas ---")
display(df_usuarios.head())

print("\n--- Últimas 5 linhas ---")
display(df_usuarios.tail())

print("\n--- Amostra aleatória de 5 linhas ---")
display(df_usuarios.sample(5))

--- Primeiras 5 linhas ---


,user_id,nome,email,data_cadastro,cidade,estado,valor_ultima_compra,data_ultimo_login
0,f47275f7-7ecb-4989-9f4f-04689d32b481,Ágatha Ferreira,henry-gabrielcamara@example.com,19/05/2024,Lopes,São Paulo,"R$ 757,15",2025-10-30 20:14:41.677555
1,3baa2972-fc7e-4adf-8dbd-79c660a77830,Enrico Cavalcante,maria-eduarda38@example.com,2025-07-30,Peixoto,SC,"R$ 404,95",2025-10-29 15:33:31.360793
2,d8f0f5d7-585d-41e6-ba1e-0a208f9ece42,Luna da Cunha,olivia01@example.net,10-12-2024,da Rosa,SC,"R$ 800,86",2025-10-27 02:51:40.654204
3,a235140a-b939-4e90-bb14-d6692e6a38fa,Benicio Porto,asilva@example.net,13/05/2024,Ribeiro do Campo,MG,NaN,2025-11-03 08:30:45.151938
4,9bf14b0f-f85c-485b-9111-b7609734d00c,Arthur Vargas,anovais@example.com,10-Aug-2024,Rezende do Norte,SP,NaN,2025-11-04 02:02:51.906484



--- Últimas 5 linhas ---


,user_id,nome,email,data_cadastro,cidade,estado,valor_ultima_compra,data_ultimo_login
210,6175491b-652e-499d-9061-c412f68af4de,Manuela Mendes,francisco95@example.com,21/07/2024,Farias,BA,"R$ 765,17",2025-10-10 04:27:07.700623
211,3ac326c6-dab1-4d7a-bc78-de817d024aa3,Julia Castro,msiqueira@example.com,2025-05-03,Casa Grande Alegre,rio de janeiro,"R$ 658,80",2025-10-24 20:01:48.478328
212,2b608eb3-1a28-4fee-b292-436ddd6bfb08,Luiz Otávio Aparecida,maria-helena92@example.com,04-Jul-2024,Novaes,BA,"R$ 195,40",2025-10-09 09:50:24.443582
213,62a74a2e-2d12-4684-af88-5adeb6562411,Melina Castro,vargasrafaela@example.com,2025-07-06,Sampaio de Vasconcelos,PR,"R$ 446,67",2025-10-07 19:52:10.195290
214,56b5f81e-ef2b-4901-beef-dabfa1feeb8c,Alexandre Aparecida,ana-sophia10@example.net,05-17-2025,da Rocha,BA,"R$ 922,20",2025-10-11 20:00:39.186498



--- Amostra aleatória de 5 linhas ---


,user_id,nome,email,data_cadastro,cidade,estado,valor_ultima_compra,data_ultimo_login
166,a8028236-a693-483e-b7f5-705cef2409d9,Vitor Gabriel Melo,agathaferreira@example.net,03-01-2025,Rocha das Pedras,BA,"R$ 313,80",2025-10-16 23:21:29.624698
133,270be768-7faf-4a38-8c28-3925bb601243,Isabelly Nogueira,breno17@example.net,04-Jun-2025,Pastor,Rio de Janeiro,"R$ 425,29",2025-10-18 02:48:35.860844
181,16521b7e-8f85-4ac8-9961-1f89487c3c50,Fernando Novaes,NaN,2025-02-10,Montenegro da Praia,PR,"R$ 845,52",2025-10-25 03:46:53.921190
9,f73db410-07b9-4443-b334-2f851b21372d,Gabriela Dias,das-nevesvitor@example.net,2025-02-27,Rocha,MG,"R$ 779,99",2025-10-25 09:45:53.765840
121,d2ecf162-13e6-4704-8d08-640d9db8e6c0,Cecília Rocha,da-conceicaopietra@example.net,11-19-2023,Nogueira da Serra,PR,"R$ 145,02",2025-10-16 01:45:49.578645


#### Usando `.info()` para um Resumo Técnico

O método `.info()` é um dos nossos melhores amigos. Ele nos dá um resumo conciso do DataFrame, incluindo:
- O número total de entradas (linhas).
- O número de colunas.
- O nome e a contagem de valores **não nulos** para cada coluna.
- O **tipo de dado (`Dtype`)** de cada coluna.
- O uso de memória.

**O que procurar aqui?**
1.  **Contagem de Não Nulos:** Se o valor for menor que o total de entradas, significa que a coluna tem dados faltantes.
2.  **Dtype:** O tipo de dado está correto? Uma coluna de valor de compra deveria ser numérica (`float64` ou `int64`), não `object` (que geralmente significa string). Uma coluna de data deveria ser `datetime64[ns]`, não `object`.

In [ ]:
# Obtendo um resumo técnico do DataFrame
df_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              215 non-null    object
 1   nome                 215 non-null    object
 2   email                196 non-null    object
 3   data_cadastro        215 non-null    object
 4   cidade               215 non-null    object
 5   estado               215 non-null    object
 6   valor_ultima_compra  197 non-null    object
 7   data_ultimo_login    215 non-null    object
dtypes: object(8)
memory usage: 13.6+ KB


#### Usando `.describe(include='all')` para Estatísticas Descritivas

Enquanto `.info()` nos dá a estrutura, `.describe()` nos dá um resumo estatístico. Usando `include='all'`, forçamos o Pandas a nos mostrar estatísticas tanto para colunas numéricas quanto para as de texto (categóricas).

- **Para colunas numéricas:** `count`, `mean` (média), `std` (desvio padrão), `min`, `max`, e os quartis (`25%`, `50%`, `75%`).
- **Para colunas de objeto/categóricas:** `count`, `unique` (número de valores únicos), `top` (valor mais frequente), e `freq` (frequência do valor mais frequente).

In [ ]:
# Obtendo um resumo estatístico de todas as colunas
df_usuarios.describe(include='all')

,user_id,nome,email,data_cadastro,cidade,estado,valor_ultima_compra,data_ultimo_login
count,215,215,196,215,215,215,197,215
unique,200,200,182,194,163,10,177,200
top,7fedad2f-41f3-48d9-ab26-005ec7943ff5,Sara Pires,gabrielacaldeira@example.net,05-17-2025,da Rosa,BA,Não informado,2025-10-23 18:52:46.667089
freq,2,2,2,3,4,48,7,2


### ✏️ Exercício 1: Análise Pós-Inspeção

Com base nas saídas dos comandos `.info()` e `.describe(include='all')`, responda na célula abaixo às seguintes perguntas:

1.  Quais colunas têm valores faltantes? A contagem de não nulos em `.info()` te deu essa resposta.
2.  A coluna `valor_ultima_compra` é do tipo correto para realizarmos cálculos (como a média)?
3.  As colunas `data_cadastro` e `data_ultimo_login` estão em um formato de data que o pandas entende nativamente?
4.  Olhando para a estatística `unique` da coluna `estado` no `.describe()`, o número parece alto ou baixo demais? O que isso pode indicar?



```
# Isto está formatado como código
```
Respostas:
1. As colunas email e valor_ultima_compra possuem valores faltantes

2. Acredito que não, porque tem strings como "R$"  e "não informado"

3. As colunas data_ cadastro e data_ultimo_login estão como object

4. Acredito que parece alto, penso que número indica que o mesmo Estado aparece escrito de formas diferentes. Nesse sentido, essas variações como maiúsculas, minúsculas etc... fazem o pandas contar cada forma como um valor diferente aumentando o total de unique


Responda aqui.

### 2.3 Tratando Dados Faltantes (Valores Nulos)

Nossa investigação revelou que as colunas `email` e `valor_ultima_compra` têm valores faltantes. Vamos lidar com eles.

In [ ]:
# Contando o número de valores nulos em cada coluna
df_usuarios.isnull().sum()

,0
user_id,0
nome,0
email,19
data_cadastro,0
cidade,0
estado,0
valor_ultima_compra,18
data_ultimo_login,0


#### Estratégias para Lidar com Dados Faltantes

Existem duas estratégias principais para lidar com dados faltantes:

1.  **Remoção:** Excluir as linhas (ou colunas) que contêm valores faltantes. É uma abordagem rápida e simples, mas tem um custo: a perda de dados. Se uma linha tem um valor importante faltando, mas as outras informações são valiosas, removê-la pode não ser o ideal.
2.  **Imputação (Preenchimento):** Preencher os valores faltantes com um valor estimado. Pode ser um valor fixo (como 0), a média, a mediana ou a moda da coluna. Esta abordagem preserva o resto dos dados da linha, mas introduz um valor que não é original.

A escolha da estratégia depende do contexto do negócio e da natureza da coluna.

#### Estratégia 1: Remover Linhas com `dropna()`

**Cenário:** Um usuário sem e-mail é de pouca utilidade para nosso e-commerce. Não podemos contatá-lo para marketing, recuperação de senha ou confirmação de pedidos. Portanto, a decisão de negócio aqui é **remover** os cadastros que não possuem um e-mail.

Usaremos `df.dropna(subset=['nome_da_coluna'])` para remover apenas as linhas onde o valor na coluna especificada é nulo.

In [ ]:
# Verificando o número de linhas antes da remoção
print(f"Número de linhas antes de remover nulos em 'email': {len(df_usuarios)}")

# Contando os nulos em 'email' para confirmar
print(f"Número de valores nulos em 'email': {df_usuarios['email'].isnull().sum()}\n")

# Removendo as linhas onde a coluna 'email' é nula
# O parâmetro 'inplace=True' modifica o DataFrame diretamente, sem precisar de reatribuição (df = df.dropna(...))
df_usuarios.dropna(subset=['email'], inplace=True)

# Verificando o número de linhas depois da remoção
print(f"Número de linhas após remover nulos em 'email': {len(df_usuarios)}")

# Confirmando que não há mais nulos em 'email'
print(f"Número de valores nulos em 'email' agora: {df_usuarios['email'].isnull().sum()}")

Número de linhas antes de remover nulos em 'email': 215
Número de valores nulos em 'email': 26

Número de linhas após remover nulos em 'email': 189
Número de valores nulos em 'email' agora: 0


#### Estratégia 2: Imputar Valores com `fillna()`

**Cenário:** A coluna `valor_ultima_compra` também tem valores nulos. No entanto, remover essas linhas significaria perder informações de usuários que, embora não tenham um valor de compra registrado, ainda são clientes cadastrados. Uma abordagem melhor é a **imputação**.

**Média vs. Mediana:** Qual valor usar para preencher?
- **Média (`mean`):** A soma de todos os valores dividida pelo número de valores. É muito sensível a *outliers* (valores extremamente altos ou baixos). Uma única compra de valor muito alto poderia inflar a média e distorcer a realidade.
- **Mediana (`median`):** O valor do meio quando todos os dados são ordenados. É robusta a outliers e, por isso, é geralmente a escolha mais segura para dados financeiros ou com distribuição assimétrica, como valores de compra.

Vamos tentar calcular a mediana e preencher os valores nulos. Mas... há um problema!

In [ ]:
try:
    mediana_compra = df_usuarios['valor_ultima_compra'].median()
    print(f"Mediana calculada: {mediana_compra}")
except TypeError as e:
    print(f"Ocorreu um erro: {e}")
    print("\nNão podemos calcular a mediana de uma coluna que não é numérica! Isso nos leva ao próximo passo.")

Ocorreu um erro: Cannot convert ['R$ 757,15' 'R$ 404,95' 'R$ 800,86' nan nan 'R$ 285,29' 'R$ 999,00'
 'R$ 953,79' 'R$ 870,53' 'R$ 779,99' nan 'R$ 164,51' 'R$ 776,21'
 'R$ 864,59' 'Não informado' 'R$ 976,25' 'R$ 85,88' nan 'R$ 501,25'
 'R$ 270,96' 'R$ 257,86' 'R$ 267,37' nan 'R$ 884,82' 'Não informado'
 'R$ 374,10' 'R$ 904,06' 'R$ 894,68' 'R$ 427,36' nan 'R$ 943,10'
 'Não informado' nan 'R$ 364,71' 'R$ 114,96' 'R$ 266,59' 'R$ 569,02'
 'R$ 737,30' 'R$ 479,12' nan 'R$ 388,54' 'R$ 651,10' 'R$ 304,78'
 'R$ 938,21' 'R$ 374,78' 'R$ 653,94' 'R$ 306,35' 'R$ 264,75' 'R$ 180,24'
 'Não informado' 'R$ 558,26' 'R$ 48,57' 'R$ 452,43' 'R$ 149,99'
 'R$ 469,55' 'R$ 805,41' 'R$ 515,33' 'R$ 525,59' 'R$ 159,73' 'R$ 485,22'
 'R$ 437,28' 'R$ 336,54' 'R$ 376,02' 'R$ 606,45' 'R$ 676,85' 'R$ 151,46'
 'R$ 765,17' 'R$ 996,65' 'R$ 758,72' 'R$ 40,89' nan nan 'R$ 871,88'
 'R$ 894,80' 'R$ 596,64' 'R$ 89,82' 'Não informado' 'R$ 395,13'
 'R$ 446,67' 'R$ 438,11' 'R$ 984,15' 'R$ 800,88' 'R$ 355,88' 'R$ 420,87'
 'R$ 63,44

### 2.4 Corrigindo Tipos de Dados

O erro acima aconteceu porque, como vimos no `.info()`, a coluna `valor_ultima_compra` é do tipo `object` (string), e não um número. Precisamos convertê-la.

#### Convertendo `valor_ultima_compra` para Numérico

Para converter, precisamos primeiro limpar a string, removendo o `R$ ` e trocando a vírgula decimal por um ponto. Depois, usamos `pd.to_numeric`.

O parâmetro `errors='coerce'` é muito útil: ele transformará qualquer valor que não possa ser convertido em um número (como a string 'Não informado') em `NaN`. Isso é ótimo, pois podemos tratar todos os problemas de uma vez só.

In [ ]:
# Passo 1: Limpar a string
# Usamos.str para aplicar métodos de string a toda a coluna
df_usuarios['valor_ultima_compra'] = df_usuarios['valor_ultima_compra'].str.replace('R$ ', '', regex=False)
df_usuarios['valor_ultima_compra'] = df_usuarios['valor_ultima_compra'].str.replace(',', '.', regex=False)

# Passo 2: Converter para tipo numérico, tratando erros
df_usuarios['valor_ultima_compra'] = pd.to_numeric(df_usuarios['valor_ultima_compra'], errors='coerce')

# Vamos verificar o tipo de dado da coluna agora
print("Tipo de dado de 'valor_ultima_compra' após conversão:")
print(df_usuarios.dtypes['valor_ultima_compra'])

# E ver como ficaram os 10 primeiros valores
print("\nValores após conversão (note os novos NaNs onde antes era 'Não informado'):")
display(df_usuarios[['nome', 'valor_ultima_compra']].head(10))

Tipo de dado de 'valor_ultima_compra' após conversão:
float64

Valores após conversão (note os novos NaNs onde antes era 'Não informado'):


,nome,valor_ultima_compra
0,Ágatha Ferreira,757.15
1,Enrico Cavalcante,404.95
2,Luna da Cunha,800.86
3,Benicio Porto,NaN
4,Arthur Vargas,NaN
5,Benício Moraes,285.29
6,José Pimenta,999.00
7,Ana Liz Sousa,953.79
8,Asafe Camargo,870.53
9,Gabriela Dias,779.99


#### Agora sim: Imputando a Mediana

Com a coluna `valor_ultima_compra` agora no formato `float64`, podemos finalmente calcular a mediana e usar `fillna()` para preencher todos os valores `NaN` (os que já existiam e os que foram criados pelo `errors='coerce'`).

In [ ]:
# Verificando nulos ANTES da imputação
print(f"Nulos em 'valor_ultima_compra' ANTES da imputação: {df_usuarios['valor_ultima_compra'].isnull().sum()}")

# 1. Calcular a mediana (agora vai funcionar!)
mediana_compra = df_usuarios['valor_ultima_compra'].median()
print(f"A mediana calculada é: R$ {mediana_compra:.2f}")

# 2. Preencher os valores nulos com a mediana
df_usuarios['valor_ultima_compra'].fillna(mediana_compra, inplace=True)

# Verificando nulos depois da imputação
print(f"Nulos em 'valor_ultima_compra' APÓS a imputação: {df_usuarios['valor_ultima_compra'].isnull().sum()}")

Nulos em 'valor_ultima_compra' ANTES da imputação: 25
A mediana calculada é: R$ 482.17
Nulos em 'valor_ultima_compra' APÓS a imputação: 0


/tmp/ipython-input-3280222993.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_usuarios['valor_ultima_compra'].fillna(mediana_compra, inplace=True)


#### Convertendo Colunas de Data

As colunas `data_cadastro` e `data_ultimo_login` também são do tipo `object`. Precisamos convertê-las para o tipo `datetime` para que possamos realizar operações com datas, como calcular a quanto tempo um usuário se cadastrou.

A função `pd.to_datetime` é extremamente poderosa. Para a `data_cadastro`, que tem vários formatos, podemos usar o argumento `format='mixed'` para que o Pandas tente adivinhar o formato correto para cada linha.

Novamente, usaremos `errors='coerce'` para converter qualquer data que não possa ser entendida em `NaT` (Not a Time), o equivalente a `NaN` para datas.

In [ ]:
# Convertendo a coluna 'data_cadastro' para datetime
# 'format="mixed"' permite que o pandas tente adivinhar múltiplos formatos
df_usuarios['data_cadastro'] = pd.to_datetime(df_usuarios['data_cadastro'], format='mixed', errors='coerce', dayfirst=False)

# A coluna 'data_ultimo_login' tem um formato mais consistente, mas ainda é object
df_usuarios['data_ultimo_login'] = pd.to_datetime(df_usuarios['data_ultimo_login'], errors='coerce')

# Vamos verificar os tipos de dados novamente com.info()
print("--- Verificação dos Dtypes após conversão de datas ---")
df_usuarios.info()

--- Verificação dos Dtypes após conversão de datas ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              215 non-null    object        
 1   nome                 215 non-null    object        
 2   email                196 non-null    object        
 3   data_cadastro        215 non-null    datetime64[ns]
 4   cidade               215 non-null    object        
 5   estado               215 non-null    object        
 6   valor_ultima_compra  215 non-null    float64       
 7   data_ultimo_login    215 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 13.6+ KB


Sucesso! As colunas `valor_ultima_compra`, `data_cadastro` e `data_ultimo_login` agora têm os tipos de dados corretos (`float64` e `datetime64[ns]`), como esperado. Agora podemos fazer operações matemáticas e de data, como calcular o tempo desde o último login ou a média de compras.

### ✏️ Exercício 2: Cálculos Pós-Conversão

Agora que os tipos de dados estão corretos, realize as seguintes tarefas em células de código separadas:

1.  Calcule o valor **médio** da coluna `valor_ultima_compra` e imprima o resultado formatado.
2.  Encontre a data do **último login mais recente** em todo o dataset .
3.  Calcule há quantos dias foi o último login do **primeiro usuário** do DataFrame.

In [ ]:
# 1. Calcule o valor médio da coluna 'valor_ultima_compra'
print(f"Valor médio última compra: R$ {df_usuarios['valor_ultima_compra'].mean():.2f}")

Valor médio última compra: R$ 513.85


In [ ]:
# 2. Encontre a data do último login mais recente
print(f"Data do último login mais recente: {df_usuarios['data_ultimo_login'].max().strftime('%d/%m/%Y %H:%M:%S')}")

Data do último login mais recente: 05/11/2025 14:36:01


In [ ]:
# 3. Calcule há quantos dias foi o último login do primeiro usuário
print(f"O último login do primeiro usuário foi há {(datetime.now() - df_usuarios.iloc[0]['data_ultimo_login']).days} dias.")

O último login do primeiro usuário foi há 5 dias.


### 2.5 Removendo Duplicatas

Dados duplicados podem distorcer análises, como a contagem de usuários únicos. Vamos verificar se existem e removê-los.

- `.duplicated().sum()`: Conta quantas linhas são duplicatas exatas de outras que já apareceram.
- `.drop_duplicates()`: Retorna um DataFrame com as duplicatas removidas.

In [ ]:
# Verificando o número de linhas duplicadas
num_duplicatas = df_usuarios.duplicated().sum()
print(f"Número de linhas duplicadas encontradas: {num_duplicatas}")

# Removendo as duplicatas
print(f"Linhas antes de remover duplicatas: {len(df_usuarios)}")
df_usuarios.drop_duplicates(inplace=True)
print(f"Linhas após remover duplicatas: {len(df_usuarios)}")

Número de linhas duplicadas encontradas: 15
Linhas antes de remover duplicatas: 215
Linhas após remover duplicatas: 200


### 2.6 Padronizando Dados Categóricos

O último passo da nossa limpeza é garantir que dados de texto (categóricos) sejam consistentes. Na nossa inspeção, suspeitamos da coluna `estado`.

Vamos usar `.unique()` para ver todos os valores distintos que a coluna possui.

In [ ]:
# Verificando os valores únicos na coluna 'estado'
print("Valores únicos em 'estado' ANTES da padronização:")
print(df_usuarios['estado'].unique())

# Criando o dicionário de mapeamento para corrigir as inconsistências
mapa_estados = {
    'São Paulo': 'SP',
    'sao paulo': 'SP',
    'Rio de Janeiro': 'RJ',
    'rio de janeiro': 'RJ'
    # Não precisamos mapear 'SP' -> 'SP' ou 'RJ' -> 'RJ', o replace ignora chaves que não encontra
}

# Aplicando a substituição
df_usuarios['estado'].replace(mapa_estados, inplace=True)

# Verificando os valores únicos novamente para confirmar a limpeza
print("\nValores únicos em 'estado' APÓS a padronização:")
print(df_usuarios['estado'].unique())

Valores únicos em 'estado' ANTES da padronização:
['São Paulo' 'SC' 'MG' 'SP' 'BA' 'rio de janeiro' 'PR' 'Rio de Janeiro'
 'sao paulo' 'RJ']

Valores únicos em 'estado' APÓS a padronização:
['SP' 'SC' 'MG' 'BA' 'RJ' 'PR']


/tmp/ipython-input-3736765885.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_usuarios['estado'].replace(mapa_estados, inplace=True)


Excelente! Agora nossa coluna `estado` está limpa e padronizada, pronta para ser usada em análises de agrupamento (`groupby`).

### 2.7 Salvando o Trabalho

Missão cumprida! Passamos por todas as etapas do nosso fluxo de trabalho de limpeza. O passo final é salvar nosso DataFrame limpo em um novo arquivo CSV. Este arquivo será o ponto de partida para futuras análises.

Vamos dar uma última olhada no nosso trabalho com `.info()` para confirmar que tudo está em ordem: sem nulos e com os tipos de dados corretos.

In [ ]:
# Verificação final
df_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              200 non-null    object        
 1   nome                 200 non-null    object        
 2   email                182 non-null    object        
 3   data_cadastro        200 non-null    datetime64[ns]
 4   cidade               200 non-null    object        
 5   estado               200 non-null    object        
 6   valor_ultima_compra  200 non-null    float64       
 7   data_ultimo_login    200 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 14.1+ KB


In [ ]:
# Salvando o DataFrame limpo em um novo arquivo CSV
df_usuarios.to_csv('usuarios_limpo.csv', index=False)

print("Arquivo 'usuarios_limpo.csv' salvo com sucesso!")

Arquivo 'usuarios_limpo.csv' salvo com sucesso!


---
## Conclusão

Parabéns! Você completou um ciclo completo de limpeza de dados. Você pegou um dataset caótico e, aplicando um método sistemático, o transformou em uma fonte de dados organizada e confiável.

**O que nós fizemos:**
- **Inspecionamos** os dados para encontrar problemas.
- **Tratamos valores faltantes** usando duas estratégias diferentes (remoção e imputação).
- **Corrigimos tipos de dados** incorretos, permitindo cálculos e operações.
- **Removemos dados duplicados** para garantir a unicidade dos registros.
- **Padronizamos dados categóricos** para permitir agrupamentos e análises consistentes.

Agora, com o arquivo `usuarios_limpo.csv` em mãos, você está pronto para a próxima aula, onde vamos explorar e visualizar esses dados para descobrir padrões e insights de negócio.